In [1]:
import pandas as pd
from tqdm import tqdm
from collections import Counter
import matplotlib.pyplot as plt
import spacy
from sklearn.model_selection import train_test_split
from fastai.text.all import *
import torch

In [2]:
# Set device to GPU if available
if torch.cuda.is_available():
    print('GPU available. Using GPU...')
    torch.device('cuda')
else:
    print('No GPU available. Using CPU...')
    torch.device('cpu')


GPU available. Using GPU...


In [3]:
df = pd.read_csv("lai-data/nationality.csv")

df

,auhtor_ID,post,nationality
0,t2_10uons,"It isn't, it's just for show. It isn't legally binding. Depends And also for most ranks it's better to play something you're good at, rather than playing someone you're bad at but is meta/countering The added risk of fire with it on the side is basically 0 Yeah technically it does increase the risk. But it's so small it's basically like being worried that a drop of water on the road will cause an accident. Aaahh yes. UK and US. The only two nations in the world Hint: other countries do also have kettles Yeah that's Apple for you Because you're basically joining the war purely for the reaso...",Finland
1,t2_10uons,"games won to games played. Games played is calculated compared to how much of the match you played. Eg. You played half match as mercy, you get 0.5 matches played to mercy. Matches won seems to be that you either win or you don't, nothing in between. So your first game as mercy, playing half the game and you won, means the winrate would be 1 (wins) / 0.5 (games played) = 2 = 200% winrate. Can't confirm if it works like this but this is my guess on how it works. I do. It's cheaper and works just as well. Considering it has hotel in the name, I'm guessing it's meant for hotels. Yeah i guess ...",Finland
2,t2_10uons,"So with that logic USA is a very shitty place to live (with ~1 shooting per million people) and literally every other nation being a better place to live in (with China being the best) Good bot It seems like a morning news show so they are more casual/chill than news normally are. Keep in mind what you see in the video is presumably the first and only thing that they know about furries, and it is understand that you would be confused. I'm guessing she's having one of those moments where when you're not allowed laugh, just makes it harder not to laugh. This clip is ~9 years old Nah it makes...",Finland
3,t2_10uons,"are referring to It does add up a lot when EA, Activision etc.. can re-release the same game each year! And also like they would be better than recommendations. It depends on many factors. Kyllä. Tuntuu paremmalta koska se käytännössä on ""ilmaista rahaa"". Olet jo valmiiksi suunnitellut elämään nykyisillä tuloilla ja yllättäen tulee sitten joku 600€ extra käytettäväksi. Objektiiviset ei ole hyvä säästämiseen mutta käytännössä on. Siinä on huomattava tunne ero jos saa 600€ yllättäen, tai jos laittaa 50€ kuukaudessa sivuun. Pitää myöskin ottaa huomioon että tuleeko sitä oikeasti säästettyä 60...",Finland
4,t2_10uons,"get another role than support Lel people here thinking 23 is some huge number that you must be a whore to achieve by 22 Of course it can happen anywhere. But the difference is that it happens far less in other places. Your logic is like saying schools are dangerous everywhere because the US has a shooting like once a week. Wouldn't be surprised. This gets way more engagement than if they'd just pour water on it. Nothing cursed about it tho ""Wow. This tool works well for its intended purpose but would not work so well for something it wasn't designed for"" Are you infuriated that someone rus...",Finland
...,...,...,...
82611,t2_57ogt82f,"and Sergeant York. WWI didn't lend it'self to films. Most of it was fought in cold wet trenches in Northern Europe. Much of the time was spent waiting for the next round of shelling. By contrast WWII was fought on every continent. This gave film makers far more opportunity to feature exotic places and scantly clad local women. There are no surviving military from WWI and an ever reducing number from WWII. So for WWI there are only people like myself who have talked with people who fought in it. Both my grandfaters talked about it, and the thing which always came up was the cold, damp condi...",United Kingdom
82612,t2_57ogt82f,"a position where a person can get short term fame by committing atrocities. I don't think clamping down on guns is the answer. Any control strong enough to make a difference would not be a

In [4]:
european_countries = ["Finland", "Germany", "Greece", "The Netherlands", "Ireland", "Spain", "United Kingdom", "Denmark", "France", "Italy", "Lithuania", "Poland", "Sweden", "Romania", "Switzerland", "Norway", "Bulgaria", "Hungary", "Belgium", "Serbia", "Austria", "Ukraine", "Croatia", "Slovenia", "Czech Republic", "Slovakia", "Cyprus", "Luxembourg", "Estonia", "Portugal", "Kosovo", "Bosnia and Herzegovina", "Iceland", "Albania", "Russia", "Montenegro", "Moldova", "Malta", "Greenland", "North Macedonia"]

df_europe = df[df["nationality"].isin(european_countries)].copy()

western = ["Finland", "Germany", "Greece", "The Netherlands", "Ireland", "Spain", "United Kingdom", "Denmark", "France", "Italy", "Sweden", "Switzerland", "Norway", "Belgium", "Austria", "Portugal", "Luxembourg", "Iceland", "Greenland", "Malta"]

df_europe.loc[:, "Western"] = df_europe["nationality"].apply(lambda x: x in western)

df_europe

,auhtor_ID,post,nationality,Western
0,t2_10uons,"It isn't, it's just for show. It isn't legally binding. Depends And also for most ranks it's better to play something you're good at, rather than playing someone you're bad at but is meta/countering The added risk of fire with it on the side is basically 0 Yeah technically it does increase the risk. But it's so small it's basically like being worried that a drop of water on the road will cause an accident. Aaahh yes. UK and US. The only two nations in the world Hint: other countries do also have kettles Yeah that's Apple for you Because you're basically joining the war purely for the reaso...",Finland,True
1,t2_10uons,"games won to games played. Games played is calculated compared to how much of the match you played. Eg. You played half match as mercy, you get 0.5 matches played to mercy. Matches won seems to be that you either win or you don't, nothing in between. So your first game as mercy, playing half the game and you won, means the winrate would be 1 (wins) / 0.5 (games played) = 2 = 200% winrate. Can't confirm if it works like this but this is my guess on how it works. I do. It's cheaper and works just as well. Considering it has hotel in the name, I'm guessing it's meant for hotels. Yeah i guess ...",Finland,True
2,t2_10uons,"So with that logic USA is a very shitty place to live (with ~1 shooting per million people) and literally every other nation being a better place to live in (with China being the best) Good bot It seems like a morning news show so they are more casual/chill than news normally are. Keep in mind what you see in the video is presumably the first and only thing that they know about furries, and it is understand that you would be confused. I'm guessing she's having one of those moments where when you're not allowed laugh, just makes it harder not to laugh. This clip is ~9 years old Nah it makes...",Finland,True
3,t2_10uons,"are referring to It does add up a lot when EA, Activision etc.. can re-release the same game each year! And also like they would be better than recommendations. It depends on many factors. Kyllä. Tuntuu paremmalta koska se käytännössä on ""ilmaista rahaa"". Olet jo valmiiksi suunnitellut elämään nykyisillä tuloilla ja yllättäen tulee sitten joku 600€ extra käytettäväksi. Objektiiviset ei ole hyvä säästämiseen mutta käytännössä on. Siinä on huomattava tunne ero jos saa 600€ yllättäen, tai jos laittaa 50€ kuukaudessa sivuun. Pitää myöskin ottaa huomioon että tuleeko sitä oikeasti säästettyä 60...",Finland,True
4,t2_10uons,"get another role than support Lel people here thinking 23 is some huge number that you must be a whore to achieve by 22 Of course it can happen anywhere. But the difference is that it happens far less in other places. Your logic is like saying schools are dangerous everywhere because the US has a shooting like once a week. Wouldn't be surprised. This gets way more engagement than if they'd just pour water on it. Nothing cursed about it tho ""Wow. This tool works well for its intended purpose but would not work so well for something it wasn't designed for"" Are you infuriated that someone rus...",Finland,True
...,...,...,...,...
82611,t2_57ogt82f,"and Sergeant York. WWI didn't lend it'self to films. Most of it was fought in cold wet trenches in Northern Europe. Much of the time was spent waiting for the next round of shelling. By contrast WWII was fought on every continent. This gave film makers far more opportunity to feature exotic places and scantly clad local women. There are no surviving military from WWI and an ever reducing number from WWII. So for WWI there are only people like myself who have talked with people who fought in it. Both my grandfaters talked about it, and the thing which always came up was the cold, damp condi...",United Kingdom,True
82612,t2_57ogt82f,"a position where a person can get short term fame by committing atrocities. I don't think clamping down on guns is the answer. Any control strong 

In [5]:
punctuation_matches = df_europe.post.str.findall(
    '[^\w\s]')  # matches any character that is not a word character(\w) or a whitespace character(\s).

tokens = df_europe.post.str.findall(
    '\w+|[^\w\s]')  # match either a sequence of word characters or any non-word, non-space character.

df_europe['token'] = tokens

tokens.head(5)

0                                                          [It, isn, ', t, ,, it, ', s, just, for, show, ., It, isn, ', t, legally, binding, ., Depends, And, also, for, most, ranks, it, ', s, better, to, play, something, you, ', re, good, at, ,, rather, than, playing, someone, you, ', re, bad, at, but, is, meta, /, countering, The, added, risk, of, fire, with, it, on, the, side, is, basically, 0, Yeah, technically, it, does, increase, the, risk, ., But, it, ', s, so, small, it, ', s, basically, like, being, worried, that, a, drop, of, water, on, the, road, will, cause, an, accident, ., Aaahh, ...]
1                                                                             [games, won, to, games, played, ., Games, played, is, calculated, compared, to, how, much, of, the, match, you, played, ., Eg, ., You, played, half, match, as, mercy, ,, you, get, 0, ., 5, matches, played, to, mercy, ., Matches, won, seems, to, be, that, you, either, win, or, you, don, ', t, ,, nothing, in, between,

In [6]:
def top_words_west_east(token_df, western=True):
    counter = Counter()
    for ix, row in token_df.iterrows():
        if western and row['Western']:
            counter += Counter(row['token'])
        elif not western and not row['Western']:
            counter += Counter(row['token'])

    top_words = set([w for w, i in counter.most_common(5000)])
    return top_words


In [7]:
# Assuming 'Western' column is boolean (True/False)
top_western_words = top_words_west_east(df_europe, western=True)

top_western_words

{'local',
 'equipment',
 'deaths',
 'cannot',
 'normally',
 'smoke',
 'engage',
 'Hitler',
 'add',
 'London',
 'land',
 'matter',
 'on',
 'Right',
 'wave',
 'etter',
 'UK',
 'damage',
 'Time',
 'Taiwan',
 'March',
 'mest',
 'up',
 'such',
 'articles',
 'sehen',
 'perform',
 'nuts',
 'speaks',
 'là',
 'gaat',
 'uncomfortable',
 'we',
 'complicated',
 'legislation',
 'near',
 '9',
 'nothing',
 'beginning',
 'answered',
 'actual',
 'generation',
 'will',
 'destroyed',
 'doesnt',
 'racism',
 'alle',
 'Jews',
 'promise',
 'summer',
 'steal',
 'mods',
 'detailed',
 'faut',
 'year',
 'house',
 'however',
 'call',
 'presence',
 'eigentlich',
 'Earth',
 'to',
 'Nordic',
 'på',
 'zumindest',
 'olisi',
 'gt',
 'Der',
 'Tesla',
 'emergency',
 'Dit',
 'mere',
 'allowing',
 'dal',
 'born',
 'hospitals',
 'ignore',
 'pronounced',
 'true',
 'couple',
 'laptop',
 'fees',
 'black',
 'Seriously',
 'schnell',
 'crying',
 'distinction',
 'Kan',
 'kan',
 'kick',
 'teacher',
 'Haha',
 'english',
 'wer',
 'go

In [8]:
top_eastern_words = top_words_west_east(df_europe, western=False)

top_eastern_words

{'local',
 'equipment',
 'deaths',
 'cannot',
 'normally',
 'smoke',
 'engage',
 'Hitler',
 'add',
 'London',
 'land',
 'matter',
 'on',
 'Right',
 'wave',
 'bombing',
 'UK',
 'damage',
 'Time',
 'hating',
 'Taiwan',
 'March',
 'up',
 'such',
 'articles',
 'perform',
 'oko',
 'speaks',
 'we',
 'complicated',
 'near',
 '9',
 'nothing',
 'beginning',
 'actual',
 'lub',
 'generation',
 'will',
 'destroyed',
 'doesnt',
 'racism',
 'Jews',
 'summer',
 'steal',
 'mods',
 'year',
 'Nisam',
 'house',
 'however',
 'ovo',
 'call',
 'presence',
 'Earth',
 'to',
 'kell',
 'gt',
 'elaborate',
 'Tesla',
 'guard',
 'emergency',
 'mere',
 'allowing',
 'born',
 'ignore',
 'Azov',
 'pronounced',
 'true',
 'couple',
 'до',
 'laptop',
 'black',
 'nič',
 'Seriously',
 '🇸',
 'crying',
 'kick',
 'kan',
 'teacher',
 'Haha',
 'english',
 'goals',
 'direction',
 'discussion',
 'Check',
 'blame',
 'web',
 '΄',
 'related',
 'brutal',
 'spam',
 'secret',
 'as',
 'lock',
 'southern',
 'crisis',
 'united',
 'toate',